In [1]:
elastic_rods_dir = '../../../elastic_rods/python/'
weaving_dir = '../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la, time
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# Reloadable libraries
import importlib

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

import vis.fields
import matplotlib.cm as cm

# Pseudosphere 1
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1/300, 1/30]
MODEL_NAME = "small_pseudo_sphere_1"
MODEL_PATH = osp.join(weaving_dir + 'normalized_objs/models/{}.obj'.format(MODEL_NAME))
SUBDIVISION_RESOLUTION = 20
SMOOTHING_WEIGHT = 10
REGULARIZATION_WEIGHT = 0
INPUT_SURFACE_PATH = osp.join(weaving_dir + 'normalized_objs/surface_models/{}.obj'.format(MODEL_NAME))
RIBBON_NAME = "{}_strip".format(MODEL_NAME)

In [2]:
def initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION, model_path = MODEL_PATH):
    l = elastic_rods.SurfaceAttractedLinkage(surface_path, useCenterline, model_path, subdivision_res, False, InterleavingType.weaving)
    l.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    l.setDoFs(l.getDoFs())
    l.set_holdClosestPointsFixed(True);
    l.set_attraction_tgt_joint_weight(0.01);
    l.attraction_weight = 100;
    return l

def initialize_normal_linkage(cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION, model_path = MODEL_PATH):
    l = elastic_rods.RodLinkage(model_path, subdivision_res, False)
    l.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    return l

def design_parameter_solve(l,regularization_weight = 0.1, smoothing_weight = 1):
    design_opts = elastic_rods.NewtonOptimizerOptions()
    design_opts.niter = 1000
    design_opts.verbose = 10
    l.set_design_parameter_config(use_restLen = True, use_restKappa = True)
    elastic_rods.designParameter_solve(l, design_opts, regularization_weight = regularization_weight, smoothing_weight = smoothing_weight)
    l.set_design_parameter_config(use_restLen = True, use_restKappa = True)

def get_linkage_eqm(l, opt, cam_param = default_camera_parameters, target_surf = None):
    elastic_rods.compute_equilibrium(l, options = opt)
    if (target_surf is None):
        view = linkage_vis.LinkageViewer(l, width=1024, height=640)
    else:
        view = linkage_vis.LinkageViewerWithSurface(l, target_surf, width=1024, height=640)
    view.setCameraParams(cam_param)
    return l, view

OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 10;
OPTS.beta = 1e-8
OPTS.niter = 100

### Generate straight linkage equilibrium for testing later

In [3]:
straight_linkage = initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)

In [4]:
view = linkage_vis.LinkageViewer(straight_linkage, width=1024, height=640)
view.setCameraParams(default_camera_parameters)
view.show()

Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='white', intensity=0.6, position=(0.0…

In [5]:
cview = linkage_vis.getColoredRodOrientationViewer(straight_linkage, width = 1024, height = 640, bottomColor=[79/255., 158/255., 246/255.], topColor=[180/255., 158/255., 119/255.])
cview.show()

Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='white', intensity=0.6, position=(0.0…

In [6]:
# straight_linkage.attraction_weight = 0
# with so(): straight_linkage, intial_view = get_linkage_eqm(straight_linkage, cam_param = default_camera_parameters, opt = OPTS)
# intial_view.show()

### Create Surface Attracted Linkage and solve for design parameters with fixed joint positions

In [7]:
with so(): linkage = initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)
save_tgt_joint_pos = linkage.jointPositions();

In [8]:
design_parameter_solve(linkage, regularization_weight = 0.0, smoothing_weight = 100)

0	1	143.506	143.506	1	1
10	0.0878466	1.61928	1.61928	1	1
20	0.018709	0.635144	0.635144	1	1
30	0.0171154	0.66563	0.66563	1	1
40	0.0120193	0.868297	0.868297	1	1
50	0.00194655	0.0171965	0.0171965	1	0


In [9]:
start_time = time.time()
linkage.attraction_weight = 1e-7
with so(): linkage, view = get_linkage_eqm(linkage, cam_param = default_camera_parameters, opt = OPTS, target_surf=INPUT_SURFACE_PATH)
print('compute equilibrium takes: ', time.time() - start_time)
view.viewOptions[view.ViewType.LINKAGE].color = 'red'
view.viewOptions[view.ViewType.LINKAGE].transparent = False
view.viewOptions[view.ViewType.SURFACE].transparent = True
view.viewOptions[view.ViewType.SURFACE].color = 'gray'
view.applyViewOptions()
view.show()

compute equilibrium takes:  1.0065181255340576


Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='white', intensity=0.6, position=(0.0…

In [10]:
cview = linkage_vis.getColoredRodOrientationViewer(linkage, width = 1024, height = 640, bottomColor=[79/255., 158/255., 246/255.], topColor=[180/255., 158/255., 119/255.])
cview.show()

Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='white', intensity=0.6, position=(0.0…

In [11]:
# camera_param = cview.getCameraParams()

In [12]:
cview.setCameraParams(camera_param)

NameError: name 'camera_param' is not defined

In [13]:
cview.update()

In [14]:
import cross_section_scaling
importlib.reload(cross_section_scaling)
cross_section_scaling.apply_density_based_cross_sections(linkage, elastic_rods.CrossSection.construct('rectangle', 2000, 0.3, 0.5 * np.array(RIBBON_CS)),
                                                                  elastic_rods.CrossSection.construct('rectangle', 2000, 0.3, 2.0 * np.array(RIBBON_CS)))
view.update()

In [15]:
# elastic_rods.compute_equilibrium(linkage, options = OPTS)
# v.update(True, linkage)

In [16]:
# Optionally configure the visualization colors/transparency
view.viewOptions[view.ViewType.LINKAGE].color = 'lightgreen'
view.viewOptions[view.ViewType.LINKAGE].transparent = False
view.viewOptions[view.ViewType.SURFACE].transparent = True
view.viewOptions[view.ViewType.SURFACE].color = 'gray'
view.applyViewOptions()

In [17]:
la.norm(linkage.gradient())

0.0008815099738448618

In [18]:
linkage.numJoints()

264

In [19]:
import linkage_optimization

In [20]:
OPTS.niter = 25
linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
useCenterline = True
optimizer = linkage_optimization.WeavingOptimization(linkage, INPUT_SURFACE_PATH, useCenterline, equilibrium_options=OPTS, pinJoint = 0, useFixedJoint = False)
optimizer.set_target_joint_position(save_tgt_joint_pos)
view.update()

In [21]:
optimizer.rl_regularization_weight = 1
optimizer.smoothing_weight = 10
optimizer.beta = 500000.0
optimizer.gamma = 1
def update_viewer():
    view.update()

In [ ]:
algorithm = linkage_optimization.WeavingOptAlgorithm.NEWTON_CG
optimizer.WeavingOptimize(algorithm, 2000, 1.0, 1e-2, update_viewer)

In [ ]:
optimizer.setLinkageAttractionWeight(1e-16)

In [ ]:
optimizer.set_holdClosestPointsFixed(False)

In [ ]:
cview = linkage_vis.getColoredRodOrientationViewer(linkage, width = 1024, height = 640, bottomColor=[79/255., 158/255., 246/255.], topColor=[180/255., 158/255., 119/255.])
cview.show()

### Apply perturbation 

In [ ]:
# linesearch_weaver = optimizer.get_linesearch_weaver()
# linesearch_weaver.setDoFs(straight_rod_dof)

In [ ]:
# perturbation_test_view = linkage_vis.LinkageViewer(linesearch_weaver, width=1024, height=640)
# perturbation_test_view.setCameraParams(default_camera_parameters)
# perturbation_test_view.show()

In [ ]:
# linesearch_weaver.attraction_weight

In [ ]:
# with so(): elastic_rods.compute_equilibrium(linesearch_weaver, options = OPTS)

In [ ]:
# perturbation_test_view.update()

In [ ]:
# export_linkage_geometry_to_obj(linkage, '{}_optimized.obj'.format(MODEL_NAME))

In [ ]:
# view.getCameraParams()

In [ ]:
# view.setCameraParams(default_camera_parameters)

In [ ]:
# import vis.fields
# import matplotlib.cm as cm
# sf = vis.fields.ScalarField(linkage, 0.5 * np.ones((linkage.numSegments(), linkage.segment(0).rod.numVertices())), colormap=cm.viridis, vmin = 0, vmax = 1)
# view.update(scalarField=sf)
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# view.setCameraParams(default_camera_parameters)

### Get laser cutting pattern

In [ ]:
# if not os.path.exists(RIBBON_NAME):
#     os.makedirs(RIBBON_NAME)
# get_all_curve_pattern(linkage, RIBBON_CS[1], SUBDIVISION_RESOLUTION, RIBBON_NAME, image_type='svg')

### Generate Output needed by Florin for Rendering

In [ ]:
ribbons = order_segments_by_ribbons(linkage)
write_linkage_ribbon_output_florin(linkage, ribbons, SUBDIVISION_RESOLUTION, RIBBON_NAME, True)